<h1>پیش‌بینی سود فروش محصولات با استفاده از مدل‌های رگرسیونی</h1>

<p dir="rtl">
در این پروژه، با استفاده از داده‌های فروش محصولات، هدف ما طراحی مدلی است که بتواند مقدار سود (<code>Profit</code>) را بر اساس ویژگی‌های مختلف سفارش، محصول، و زمان پیش‌بینی کند. این مدل می‌تواند به تصمیم‌گیری‌های استراتژیک شرکت‌ها در زمینه فروش، بازاریابی و مدیریت موجودی کمک کند.
</p>


<h2>وارد کردن کتابخانه‌ها و مدل‌ها</h2>

<p dir="rtl">
در این بخش، کتابخانه‌های مورد نیاز برای پیش‌پردازش داده‌ها، تقسیم داده‌ها، آموزش مدل‌های رگرسیونی، و ارزیابی عملکرد مدل‌ها وارد شده‌اند.
</p>



In [118]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import pickle

<h2>بارگذاری و بررسی اولیه داده‌ها</h2>

<p dir="rtl">
در این مرحله، داده‌ها از فایل <code>phase3_profit_prediction_dataset.csv</code> بارگذاری شده و ساختار کلی آن بررسی می‌شود. این بررسی شامل مشاهده نمونه‌ای از داده‌ها، نوع داده‌ها در هر ستون، و تعداد مقادیر یکتای هر ستون است.
</p>

<p dir="rtl">
هدف از این مرحله، آشنایی اولیه با ساختار داده‌ها، شناسایی ستون‌های متنی، عددی، و بررسی وجود مقادیر پرت یا تکراری است. این اطلاعات در طراحی مراحل بعدی پیش‌پردازش و انتخاب مدل بسیار مؤثر خواهد بود.
</p>


In [84]:
df = pd.read_csv('phase3_profit_prediction_dataset.csv')

In [85]:
df.head()

,Profit,Discount,ProfitMargin,Quantity,Sales,SalesAfterDiscount,Shipping Cost,Ship Mode,Category,CategorySubcategory,...,Order Priority,Region,Country,City,DayOfWeek,IsWeekend,Day,Month,Year,Segment
0,-6599.979980,0.7,'-1.466668743,5,4499.979980,1349.994048,451.630005,Standard Class,Technology,Technology - Machines,...,Low,East,United States,Lancaster,3,False,26,11,2013,Consumer
1,-3839.989990,0.5,'-0.47999995,4,7999.979980,3999.989990,674.820007,Same Day,Technology,Technology - Machines,...,High,South,United States,Burlington,4,False,5,11,2014,Corporate
2,-3701.889893,0.8,'-1.700001727,8,2177.580078,435.515990,172.070007,Standard Class,Office Supplies,Office Supplies - Binders,...,Medium,Central,United States,San Antonio,3,False,26,7,2011,Consumer
3,-3399.979980,0.7,'-1.333330716,5,2549.989990,764.997027,120.309998,Standard Class,Technology,Technology - Machines,...,Medium,West,United States,Louisville,6,True,18,4,2014,Home Office
4,-3059.820068,0.5,'-0.900037106,12,3399.659912,1699.829956,286.869995,Standard Class,Office Supplies,Office Supplies - Appliances,...,Medium,South,Portugal,Lisbon,2,False,10,6,2013,Corporate


In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Profit               30000 non-null  float64
 1   Discount             30000 non-null  float64
 2   ProfitMargin         30000 non-null  object 
 3   Quantity             30000 non-null  int64  
 4   Sales                30000 non-null  float64
 5   SalesAfterDiscount   30000 non-null  float64
 6   Shipping Cost        30000 non-null  float64
 7   Ship Mode            30000 non-null  object 
 8   Category             30000 non-null  object 
 9   CategorySubcategory  30000 non-null  object 
 10  Product Name         30000 non-null  object 
 11  IsReturned           30000 non-null  object 
 12  Market               30000 non-null  object 
 13  Order Priority       30000 non-null  object 
 14  Region               30000 non-null  object 
 15  Country              30000 non-null 

In [91]:
df.nunique().sort_values()

IsReturned                 2
IsWeekend                  2
Segment                    3
Category                   3
Ship Mode                  4
Year                       4
Order Priority             4
Market                     7
DayOfWeek                  7
Month                     12
Region                    13
Quantity                  14
CategorySubcategory       17
Discount                  27
Day                       31
Country                  142
City                    3197
Shipping Cost           9235
Sales                  13674
Profit                 14076
ProfitMargin           15925
SalesAfterDiscount     16593
dtype: int64

<h2>پیش‌پردازش داده‌ها و آماده‌سازی برای مدل‌سازی</h2>

<p dir="rtl">
در این مرحله، داده‌ها برای آموزش مدل‌های رگرسیونی آماده‌سازی می‌شوند. این شامل پاک‌سازی مقادیر رشته‌ای، تبدیل ویژگی‌های متنی به عددی، حذف ستون‌های غیرمفید، و نرمال‌سازی ویژگی‌های عددی است.
</p>

<p dir="rtl">
مراحل انجام‌شده:
</p>
<ul dir="rtl">
  <li>پاک‌سازی ستون <code>ProfitMargin</code> از کاراکترهای اضافی و تبدیل به عدد</li>
  <li>حذف ستون <code>Product Name</code> به‌دلیل یکتایی بالا و بی‌اثر بودن در مدل</li>
  <li>تبدیل ستون <code>IsReturned</code> به مقدار عددی (1 برای Yes، 0 برای No)</li>
  <li>تبدیل ستون‌های دسته‌ای با تعداد کلاس کم به One-Hot Encoding</li>
  <li>کاهش کلاس‌های زیاد در ستون‌های <code>Country</code> و <code>City</code> به ۱۰ مقدار پرتکرار و تبدیل بقیه به "Other"</li>
  <li>نرمال‌سازی ویژگی‌های عددی با <code>RobustScaler</code> برای کاهش تأثیر داده‌های پرت</li>
</ul>


In [87]:
df['ProfitMargin'] = (
    df['ProfitMargin']
    .astype(str)
    .str.replace("'", '', regex=False)
    .str.replace('"', '', regex=False)
    .str.replace('%', '', regex=False)
    .str.strip()
    .replace('', np.nan)
    .astype(float)
)

In [88]:
print(df['ProfitMargin'].describe())
print("تعداد مقادیر NaN:", df['ProfitMargin'].isna().sum())

count    30000.000000
mean        -0.111090
std          0.531228
min         -4.733558
25%         -0.267225
50%          0.039931
75%          0.236111
max          0.500000
Name: ProfitMargin, dtype: float64
تعداد مقادیر NaN: 0


In [90]:
df = df.drop(columns=["Product Name"])
print(df.columns.tolist())

['Profit', 'Discount', 'ProfitMargin', 'Quantity', 'Sales', 'SalesAfterDiscount', 'Shipping Cost', 'Ship Mode', 'Category', 'CategorySubcategory', 'IsReturned', 'Market', 'Order Priority', 'Region', 'Country', 'City', 'DayOfWeek', 'IsWeekend', 'Day', 'Month', 'Year', 'Segment']


In [92]:
df['IsReturned'] = df['IsReturned'].map({'Yes': 1, 'No': 0})
df['IsReturned'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 30000 entries, 0 to 29999
Series name: IsReturned
Non-Null Count  Dtype
--------------  -----
30000 non-null  int64
dtypes: int64(1)
memory usage: 234.5 KB


In [93]:
low_cardinality = ['Ship Mode', 'Category', 'CategorySubcategory', 'Market', 'Order Priority', 'Region', 'Segment']
df = pd.get_dummies(df, columns=low_cardinality, drop_first=True)

In [94]:
for col in ['Country', 'City']:
    top_values = df[col].value_counts().nlargest(10).index
    df[col] = df[col].apply(lambda x: x if x in top_values else 'Other')

df = pd.get_dummies(df, columns=['Country', 'City'], drop_first=True)

In [95]:
num_cols = ['Discount', 'Quantity', 'Sales', 'SalesAfterDiscount', 'Shipping Cost', 'ProfitMargin', 'DayOfWeek', 'Day',
            'Month', 'Year']

scaler = RobustScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

<h2>تقسیم داده‌ها به آموزش و تست</h2>

<p dir="rtl">
در این مرحله، ابتدا ویژگی‌ها (<code>X</code>) از ستون هدف (<code>Profit</code>) جدا می‌شوند. سپس داده‌ها به دو بخش آموزش (۸۰٪) و تست (۲۰٪) تقسیم می‌شوند تا بتوان عملکرد مدل را روی داده‌های دیده‌نشده ارزیابی کرد.
</p>

<p dir="rtl">
پارامترهای استفاده‌شده:
</p>
<ul dir="rtl">
  <li><code>test_size=0.2</code>: ۲۰٪ از داده‌ها برای تست کنار گذاشته می‌شود</li>
  <li><code>random_state=42</code>: برای تکرارپذیری نتایج، یک مقدار ثابت برای تصادفی‌سازی استفاده شده است</li>
</ul>


In [96]:
X = df.drop(columns=['Profit'])
y = df['Profit']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [97]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24000 entries, 21753 to 23654
Data columns (total 76 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Discount                                          24000 non-null  float64
 1   ProfitMargin                                      24000 non-null  float64
 2   Quantity                                          24000 non-null  float64
 3   Sales                                             24000 non-null  float64
 4   SalesAfterDiscount                                24000 non-null  float64
 5   Shipping Cost                                     24000 non-null  float64
 6   IsReturned                                        24000 non-null  int64  
 7   DayOfWeek                                         24000 non-null  float64
 8   IsWeekend                                         24000 non-null  bool   
 9   Day               

<h2>تعریف مدل‌ها و تنظیمات هایپرپارامترها</h2>

<p dir="rtl">
در این مرحله، مجموعه‌ای از مدل‌های رگرسیونی تعریف شده‌اند که هرکدام با مجموعه‌ای از هایپرپارامترها برای تنظیم بهینه عملکردشان آماده شده‌اند. این مدل‌ها در مرحله بعد با استفاده از <code>GridSearchCV</code> آموزش داده خواهند شد.
</p>

<p dir="rtl">
هر مدل دارای مجموعه‌ای از پارامترهاست که در مرحله تنظیم با <code>GridSearchCV</code> بررسی می‌شوند تا بهترین ترکیب برای بیشترین دقت انتخاب شود.
</p>


In [98]:

models = {
    'Ridge': {
        'model': Ridge(),
        'params': {
            'alpha': [0.01, 0.1, 1, 10, 100]
        }
    },
    'ElasticNet': {
        'model': ElasticNet(),
        'params': {
            'alpha': [0.01, 0.1, 1, 10],
            'l1_ratio': [0.1, 0.5, 0.9],
            'max_iter': [1000, 5000],
            'tol': [1e-3, 1e-4]
        }
    },
    'SVR': {
        'model': SVR(),
        'params': {
            'C': [0.1, 1, 10],
            'kernel': ['linear', 'rbf'],
            'epsilon': [0.01, 0.1]
        }
    },
    'GradientBoosting': {
        'model': GradientBoostingRegressor(),
        'params': {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.1],
            'max_depth': [3, 5]
        }
    },
    'RandomForest': {
        'model': RandomForestRegressor(),
        'params': {
            'n_estimators': [100, 200],
            'max_depth': [5, 10, None],
            'min_samples_split': [2, 5]
        }
    },
    'XGBoost': {
        'model': XGBRegressor(),
        'params': {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.1],
            'max_depth': [3, 5],
            'reg_alpha': [0, 0.1],
            'reg_lambda': [1, 5]
        }
    }
}

<h2>آموزش مدل‌ها با GridSearchCV و ارزیابی روی داده‌های تست</h2>

<p dir="rtl">
در این مرحله، با استفاده از <code>GridSearchCV</code>، بهترین ترکیب هایپرپارامترها برای هر مدل پیدا می‌شود. سپس مدل نهایی روی داده‌های تست ارزیابی می‌شود تا عملکرد واقعی آن سنجیده شود.
</p>
<p dir="rtl">
مراحل انجام‌شده:
</p>
<ul dir="rtl">
  <li>استفاده از <code>GridSearchCV</code> برای جستجوی بهترین ترکیب هایپرپارامترها با معیار <code>R²</code></li>
  <li>استفاده از <code>cv=2</code> برای اعتبارسنجی متقاطع سریع</li>
  <li>ذخیره بهترین مدل، امتیاز و پارامترها در دیکشنری <code>best_models</code></li>
  <li>پیش‌بینی روی داده‌های تست و محاسبه <code>R²</code> نهایی برای هر مدل</li>
</ul>


In [99]:
best_models = {}

for name, config in models.items():
    grid = GridSearchCV(config['model'], config['params'], cv=2, scoring='r2', n_jobs=-1)
    grid.fit(X_train, y_train)
    best_models[name] = {
        'best_estimator': grid.best_estimator_,
        'best_score': grid.best_score_,
        'best_params': grid.best_params_
    }
    print(f"{name} - بهترین r2: {grid.best_score_:.4f} با پارامترها: {grid.best_params_}")

Ridge - بهترین r2: 0.5859 با پارامترها: {'alpha': 100}
ElasticNet - بهترین r2: 0.6140 با پارامترها: {'alpha': 0.1, 'l1_ratio': 0.1, 'max_iter': 1000, 'tol': 0.001}
SVR - بهترین r2: 0.5878 با پارامترها: {'C': 0.1, 'epsilon': 0.01, 'kernel': 'linear'}
GradientBoosting - بهترین r2: 0.9374 با پارامترها: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
RandomForest - بهترین r2: 0.9174 با پارامترها: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 100}
XGBoost - بهترین r2: 0.9099 با پارامترها: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'reg_alpha': 0, 'reg_lambda': 1}


In [120]:
print("\nارزیابی روی test set:")
for name, model_info in best_models.items():
    test_pred = model_info['best_estimator'].predict(X_test)
    test_r2 = r2_score(y_test, test_pred)
    print(f"{name} - R² روی test: {test_r2:.4f}")


ارزیابی روی test set:
Ridge - R² روی test: 0.6066
ElasticNet - R² روی test: 0.6246
SVR - R² روی test: 0.6893
GradientBoosting - R² روی test: 0.9692
RandomForest - R² روی test: 0.9847
XGBoost - R² روی test: 0.9667


<h2>انتخاب مدل نهایی</h2>

<p dir="rtl">
پس از آموزش و ارزیابی چندین مدل رگرسیونی شامل <strong>Ridge</strong>، <strong>ElasticNet</strong>، <strong>SVR</strong>، <strong>GradientBoosting</strong>، <strong>RandomForest</strong> و <strong>XGBoost</strong>، در نهایت مدل <strong>GradientBoostingRegressor</strong> به‌عنوان مدل نهایی انتخاب شد.
</p>

<h3>دلایل انتخاب:</h3>
<ul dir="rtl">
  <li><strong>دقت بالا:</strong> مقدار R² روی داده‌های تست برابر با <code>0.9692</code> بود که نشان‌دهنده قدرت پیش‌بینی بسیار خوب مدل است.</li>
  <li><strong>تعادل بین دقت و پیچیدگی:</strong> در مقایسه با مدل‌های RandomForest و XGBoost که دقت مشابهی داشتند، GradientBoosting سبک‌تر و سریع‌تر است.</li>
  <li><strong>قابلیت تعمیم‌پذیری:</strong> مدل روی داده‌های تست عملکرد پایداری داشت و دچار overfitting نشد.</li>
  <li><strong>قابل تفسیر بودن:</strong> امکان استخراج اهمیت ویژگی‌ها و تحلیل رفتار مدل وجود دارد.</li>
</ul>

<p dir="rtl">
در نتیجه، مدل <strong>GradientBoostingRegressor</strong> با پارامترهای بهینه‌شده زیر به‌عنوان مدل نهایی انتخاب شد:
</p>

<pre>
GradientBoostingRegressor(
    learning_rate=0.1,
    max_depth=3,
    n_estimators=100
)
</pre>


<h2>ارزیابی نهایی مدل GradientBoosting</h2>

<p dir="rtl">
پس از انتخاب مدل نهایی (GradientBoostingRegressor)، عملکرد آن روی داده‌های تست با استفاده از متریک‌های کلیدی ارزیابی می‌شود.
</p>

<p dir="rtl">
متریک‌های استفاده‌شده:
</p>
<ul dir="rtl">
  <li><strong>R² (ضریب تعیین):</strong> نشان می‌دهد چه درصدی از واریانس داده‌ها توسط مدل توضیح داده شده است. هرچه به ۱ نزدیک‌تر، بهتر.</li>
  <li><strong>MAE (میانگین قدر مطلق خطا):</strong> میانگین فاصله پیش‌بینی‌ها از مقادیر واقعی. قابل تفسیر و مقاوم به داده‌های پرت.</li>
  <li><strong>MSE (میانگین مربعات خطا):</strong> میانگین مربع خطاها. به خطاهای بزرگ حساس‌تر است و برای بررسی دقت کلی مدل مفید است.</li>
</ul>


In [112]:

final_model = best_models['GradientBoosting']['best_estimator']
y_pred = final_model.predict(X_test)

print("ارزیابی نهایی مدل GradientBoosting:")
print("R²:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))





ارزیابی نهایی مدل GradientBoosting:
R²: 0.9691952349923271
MAE: 5.078990644411901
MSE: 340.6760326742739


<h2>تحلیل اهمیت ویژگی‌ها (Feature Importance)</h2>

<p dir="rtl">
مدل GradientBoostingRegressor این قابلیت را دارد که اهمیت نسبی هر ویژگی را در پیش‌بینی خروجی محاسبه کند. این تحلیل به ما کمک می‌کند تا بفهمیم کدام ویژگی‌ها بیشترین تأثیر را در پیش‌بینی سود داشته‌اند.
</p>

<p dir="rtl">
در این کد:
</p>
<ul dir="rtl">
  <li>ویژگی‌ها و میزان اهمیت آن‌ها از مدل استخراج می‌شوند.</li>
  <li>با استفاده از <code>DataFrame</code> مرتب‌سازی انجام می‌شود تا مهم‌ترین ویژگی‌ها در صدر قرار گیرند.</li>
  <li>در نهایت، ۲۰ ویژگی برتر چاپ می‌شوند که بیشترین نقش را در پیش‌بینی سود ایفا کرده‌اند.</li>
</ul>

<p dir="rtl">
این تحلیل می‌تواند برای تصمیم‌گیری‌های تجاری، انتخاب ویژگی‌های کلیدی، و بهینه‌سازی مدل‌های آینده بسیار مفید باشد.
</p>


In [116]:
importances = final_model.feature_importances_
features = X_test.columns

importance_df = pd.DataFrame({'features': features, 'importances': importances})
importance_df = importance_df.sort_values(by='importances', ascending=False)
print(importance_df.head(20))


                                     features  importances
3                                       Sales     0.479952
1                                ProfitMargin     0.470259
0                                    Discount     0.020279
2                                    Quantity     0.011118
40                         Order Priority_Low     0.005836
4                          SalesAfterDiscount     0.002971
5                               Shipping Cost     0.002754
19     CategorySubcategory_Furniture - Tables     0.001869
65                      Country_United States     0.001567
38                                  Market_US     0.001357
31  CategorySubcategory_Technology - Machines     0.000929
39                        Order Priority_High     0.000837
9                                         Day     0.000166
10                                      Month     0.000043
7                                   DayOfWeek     0.000024
13                     Ship Mode_Second Class     0.0000

<h2>تعریف تابع آماده‌سازی داده جدید برای پیش‌بینی</h2>

<p dir="rtl">
برای استفاده عملی از مدل نهایی، تابعی تعریف شده است که داده‌های جدید را دقیقاً مطابق مراحل پیش‌پردازش انجام‌شده روی داده‌های آموزش، آماده‌سازی می‌کند. این تابع شامل حذف ستون‌های غیرضروری، تبدیل ویژگی‌های متنی به عددی، کاهش کلاس‌های زیاد، و نرمال‌سازی ویژگی‌های عددی است.
</p>

<p dir="rtl">
این تابع تضمین می‌کند که داده‌های جدید دقیقاً با همان ساختار و مقیاس‌دهی وارد مدل شوند، تا پیش‌بینی‌ها معتبر و قابل اعتماد باشند.
</p>


In [ ]:
def preprocess_input_data(new_df):
    if 'Product Name' in new_df.columns:
        new_df = new_df.drop(columns=['Product Name'])

    new_df['IsReturned'] = new_df['IsReturned'].map({'Yes': 1, 'No': 0})
    new_df['IsWeekend'] = new_df['IsWeekend'].astype(int)

    low_cardinality = ['Ship Mode', 'Category', 'CategorySubcategory', 'Market', 'Order Priority', 'Region', 'Segment']
    new_df = pd.get_dummies(new_df, columns=low_cardinality, drop_first=True)

    for col in ['Country', 'City']:
        top_values = ['United States', 'India', 'Australia', 'Germany', 'France', 'China', 'Mexico', 'Brazil', 'Canada', 'United Kingdom']
        new_df[col] = new_df[col].apply(lambda x: x if x in top_values else 'Other')

    new_df = pd.get_dummies(new_df, columns=['Country', 'City'], drop_first=True)

    num_cols = ['Discount', 'Quantity', 'Sales', 'SalesAfterDiscount', 'Shipping Cost', 'ProfitMargin',
                'DayOfWeek', 'Day', 'Month', 'Year']
    scaler = RobustScaler()
    new_df[num_cols] = scaler.fit_transform(new_df[num_cols])

    return new_df


<h2>ذخیره‌سازی مدل نهایی با pickle</h2>

<p dir="rtl">
برای استفاده مجدد از مدل آموزش‌دیده بدون نیاز به آموزش دوباره، آن را با استفاده از کتابخانه <code>pickle</code> در قالب فایل باینری ذخیره می‌کنیم.
</p>

<p dir="rtl">
در این کد:
</p>
<ul dir="rtl">
  <li><code>'gradient_boosting_profit_model.pkl'</code> نام فایلی است که مدل در آن ذخیره می‌شود. می‌توان این نام را به دلخواه تغییر داد.</li>
  <li><code>'wb'</code> به معنای "نوشتن باینری" است؛ یعنی فایل به‌صورت باینری باز می‌شود تا داده‌ها در آن ذخیره شوند.</li>
  <li><code>pickle.dump(...)</code> مدل آموزش‌دیده را در فایل مشخص‌شده ذخیره می‌کند.</li>
</ul>

<p dir="rtl">
این فایل را می‌توان در آینده بارگذاری کرد و مستقیماً برای پیش‌بینی استفاده نمود، بدون نیاز به آموزش مجدد مدل.البته باید داده ها با تابع بالا آماده بشن بعد به مدل داده بشن
</p>


In [119]:
with open('gradient_boosting_profit_model.pkl', 'wb') as f:
    pickle.dump(final_model, f)